In [31]:
# The dataset captures detailed transactional sales data across multiple countries, market segments and product lines, including financial metrics such as units sold, gross sales, discounts, COGS and profit. 
# It spans the years 2013–2014 and is structured to support analysis of pricing strategies, profitability and regional performance trends.
import pandas as pd # Import the pandas library 

# --- 1. Import dataset ---
f_df = pd.read_csv("Financials.csv")

In [33]:
# --- 2. Remove "$" signs, commas and convert numeric columns ---
numeric_columns = [
    "Units Sold", "Manufacturing Price", "Sale Price", "Gross Sales",
    "Discounts", "Sales", "COGS", "Profit"
]

In [35]:
# Loop through each column in the list of numeric columns
for columns in numeric_columns:
    
    f_df[columns] = (
        f_df[columns]
        .astype(str) # Ensure all values are treated as strings for uniform processing
        .str.replace(r"[\$,]", "", regex=True) # Remove dollar signs and commas using regex
        .str.strip() # Remove leading/trailing whitespace
        .replace({"-": None, "": None}) # Replace dashes and empty strings with None (missing values)
    )

    f_df[columns] = pd.to_numeric(f_df[columns], errors="coerce") # - 'errors="coerce"' ensures that any non-convertible values become NaN

In [37]:
# --- 3. Handle missing values in Discount Band ---
f_df["Discount Band"] = f_df["Discount Band"].replace("None", pd.NA)

In [39]:
# --- 4. Convert Date → datetime, extract Month + Year ---
f_df["Date"] = pd.to_datetime(f_df["Date"], errors="coerce") # Convert the 'Date' column from string format to pandas datetime format whilst 'errors="coerce"' ensures that any invalid or unparseable dates are converted to NaT (missing datetime)
f_df["Month"] = f_df["Date"].dt.month_name() # Extract the full month name from the parsed 'Date' column
f_df["Year"] = f_df["Date"].dt.year # Extract the year from the parsed 'Date' column

In [41]:
# --- 5. Standardise categories (strip whitespace, title case) ---
categorical_columns = ["Segment", "Country", "Product", "Discount Band", "Month Name"] # Define the list of categorical columns to clean
for col in categorical_columns: # Loop through each categorical column to clean text formatting 
    f_df[col] = f_df[col].astype(str).str.strip().str.title() # Ensure all values are treated as strings, remove leading and trailing whitespace and convert to title case

# --- 6. Check for duplicates ---
duplicates = f_df.duplicated().sum() # Count the number of duplicate rows in the dataset
f_df = f_df.drop_duplicates() # Drop all exact duplicate rows from the dataset
print(f"Removed {duplicates} duplicate rows.") # Print the number of duplicates removed for audit and transparency

Removed 0 duplicate rows.


In [43]:
# --- 7. Metadata summary ---
# Numeric summary
numeric_summary = f_df[numeric_columns].describe()

# Categorical summary
categorical_summary = {col: f_df[col].value_counts() for col in categorical_columns}

In [45]:
# --- 8. Data dictionary ---
data_dictionary = {
    "Segment": "Market segment (Government, Midmarket, Enterprise, Channel Partners, Small Business)",
    "Country": "Country where the transaction occurred",
    "Product": "Product sold (Carretera, Montana, Paseo, Velo, VTT, Amarilla)",
    "Discount Band": "Discount category applied (None, Low, etc.)",
    "Units Sold": "Number of units sold",
    "Manufacturing Price": "Cost to manufacture one unit",
    "Sale Price": "Price at which product was sold per unit",
    "Gross Sales": "Total sales before discounts (Units Sold × Sale Price)",
    "Discounts": "Total discount applied",
    "Sales": "Net sales after discounts",
    "COGS": "Cost of goods sold (Units Sold × Manufacturing Price)",
    "Profit": "Net profit (Sales – COGS)",
    "Date": "Transaction date",
    "Month Number": "Month of transaction (1–12)",
    "Month Name": "Month name (January–December)",
    "Year": "Year of transaction",
    "Month": "Extracted month name from Date"
}

# --- 9. Outputs ---
print("\n--- Numeric Summary ---") # Print the numeric summary statistics for all numeric columns
print(numeric_summary)

print("\n--- Categorical Summary ---") # Print the frequency distribution for each categorical column
for col, summary in categorical_summary.items():
    print(f"\n{col}:\n{summary}")

print("\n--- Data Dictionary ---") # Print the data dictionary in a readable format
for k, v in data_dictionary.items():
    print(f"{k}: {v}")


--- Numeric Summary ---
        Units Sold  Manufacturing Price  Sale Price   Gross Sales  \
count   700.000000           700.000000  700.000000  7.000000e+02   
mean   1608.294286            96.477143  118.428571  1.827594e+05   
std     867.427859           108.602612  136.775515  2.542623e+05   
min     200.000000             3.000000    7.000000  1.799000e+03   
25%     905.000000             5.000000   12.000000  1.739175e+04   
50%    1542.500000            10.000000   20.000000  3.798000e+04   
75%    2229.125000           250.000000  300.000000  2.790250e+05   
max    4492.500000           260.000000  350.000000  1.207500e+06   

           Discounts         Sales           COGS         Profit  
count     647.000000  7.000000e+02     700.000000     637.000000  
mean    14227.586198  1.696091e+05  145475.211429   27741.010267  
std     23562.833079  2.367263e+05  203865.506118   43059.332905  
min        18.410000  1.655080e+03     918.000000     285.600000  
25%      1061.0000